In [34]:
import os
import pandas as pd

# get all filenames in the Datasets directory
csv_filenames = os.listdir(path = 'Datasets')

# sort by name ascending
csv_filenames.sort()

display(csv_filenames)

['Data_of_Attack_Back.csv',
 'Data_of_Attack_Back_BufferOverflow.csv',
 'Data_of_Attack_Back_FTPWrite.csv',
 'Data_of_Attack_Back_GuessPassword.csv',
 'Data_of_Attack_Back_NMap.csv',
 'Data_of_Attack_Back_Neptune.csv',
 'Data_of_Attack_Back_Normal.csv',
 'Data_of_Attack_Back_PortSweep.csv',
 'Data_of_Attack_Back_RootKit.csv',
 'Data_of_Attack_Back_Satan.csv',
 'Data_of_Attack_Back_Smurf.csv']

In [46]:
# read data from the filenames stored in csv_file_dct
dfs: dict[str, pd.DataFrame] = dict({file: pd.read_csv(f'Datasets/{file}') for file in csv_filenames})

In [39]:
# check the number of tables
print(f'number of tables: {len(dfs)}')

number of tables: 11


In [45]:
# view some records in each table data
for key, value in dfs.items():
    print(f'{key}:')
    display(value.head(10))

Data_of_Attack_Back.csv:


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0.0,0,0,0.00,0.5454,0.08314,0,0,0,0.2,...,0.001,0.001,0.1,0,0.100,0,0.0,0.0,0.000,0.000
1,0.0,0,0,0.00,0.5454,0.08314,0,0,0,0.2,...,0.002,0.002,0.1,0,0.050,0,0.0,0.0,0.000,0.000
2,0.0,0,0,0.00,0.5454,0.08314,0,0,0,0.2,...,0.003,0.003,0.1,0,0.033,0,0.0,0.0,0.000,0.000
3,0.0,0,0,0.00,0.5454,0.08314,0,0,0,0.2,...,0.004,0.004,0.1,0,0.025,0,0.0,0.0,0.000,0.000
4,0.0,0,0,0.00,0.5454,0.08314,0,0,0,0.2,...,0.005,0.005,0.1,0,0.020,0,0.0,0.0,0.000,0.000
5,0.0,0,0,0.00,0.5454,0.08314,0,0,0,0.2,...,0.006,0.006,0.1,0,0.017,0,0.0,0.0,0.000,0.000
6,0.0,0,0,0.08,0.5454,0.08314,0,0,0,0.2,...,0.007,0.007,0.1,0,0.014,0,0.0,0.0,0.014,0.014
7,0.0,0,0,0.00,0.5454,0.08314,0,0,0,0.2,...,0.008,0.008,0.1,0,0.012,0,0.0,0.0,0.012,0.012
8,0.0,0,0,0.00,0.5454,0.08314,0,0,0,0.2,...,0.009,0.009,0.1,0,0.011,0,0.0,0.0,0.011,0.011
9,0.0,0,0,0.00,0.5454,0.08314,0,0,0,0.2,...,0.010,0.010,0.1,0,0.010,0,0.0,0.0,0.010,0.010


Data_of_Attack_Back_BufferOverflow.csv:


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0.0184,0,0.05,0.0,0.01511,0.02957,0,0,0,0.3,...,0.001,0.003,0.1,0,0.10,0.067,0,0,0.0,0.0
1,0.0305,0,0.05,0.0,0.01735,0.02766,0,0,0,0.3,...,0.002,0.004,0.1,0,0.05,0.050,0,0,0.0,0.0
2,0.0150,0,0.05,0.0,0.01587,0.06707,0,0,0,0.1,...,0.001,0.001,0.1,0,0.10,0.000,0,0,0.0,0.0
3,0.0060,0,0.05,0.0,0.02328,0.04551,0,0,0,0.3,...,0.001,0.001,0.1,0,0.10,0.000,0,0,0.0,0.0
4,0.0158,0,0.05,0.0,0.01567,0.03095,0,0,0,0.3,...,0.002,0.002,0.1,0,0.05,0.000,0,0,0.0,0.0
5,0.0113,0,0.05,0.0,0.06274,0.16771,0,0,0,0.5,...,0.001,0.001,0.1,0,0.10,0.000,0,0,0.0,0.0
6,0.0053,0,0.05,0.0,0.02628,0.03860,0,0,0,0.3,...,0.002,0.002,0.1,0,0.05,0.000,0,0,0.0,0.0
7,0.0000,0,0.14,0.0,0.00000,0.05690,0,0,0,0.0,...,0.001,0.001,0.1,0,0.10,0.000,0,0,0.0,0.0
8,0.0000,0,0.14,0.0,0.00000,0.05828,0,0,0,0.0,...,0.002,0.002,0.1,0,0.10,0.000,0,0,0.0,0.0
9,0.0000,0,0.14,0.0,0.00000,0.05020,0,0,0,0.0,...,0.003,0.003,0.1,0,0.10,0.000,0,0,0.0,0.0


Data_of_Attack_Back_FTPWrite.csv:


...

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0.0060,0.00,0.05,0,0.00086,0.00183,0,0,0.0,0.0,...,0.255,0.001,0.000,0.002,0.00,0.00,0,0.000,0.000,0.000
1,0.0060,0.00,0.05,0,0.00090,0.00233,0,0,0.0,0.0,...,0.255,0.002,0.001,0.002,0.00,0.00,0,0.000,0.000,0.000
2,0.0708,0.00,0.05,0,0.01727,0.24080,0,0,0.0,0.0,...,0.255,0.003,0.001,0.002,0.00,0.00,0,0.000,0.000,0.000
3,0.0021,0.00,0.07,0,0.00089,0.00345,0,0,0.0,0.1,...,0.255,0.001,0.000,0.002,0.00,0.00,0,0.000,0.000,0.000
4,0.0098,0.00,0.05,0,0.00621,0.08356,0,0,0.1,0.1,...,0.255,0.004,0.002,0.002,0.00,0.00,0,0.000,0.000,0.000
5,0.0000,0.00,0.14,0,0.00000,0.05636,0,0,0.0,0.0,...,0.001,0.041,0.100,0.000,0.10,0.01,0,0.000,0.000,0.000
6,0.0061,0.00,0.05,0,0.00294,0.03929,0,0,0.0,0.0,...,0.255,0.004,0.002,0.002,0.00,0.00,0,0.025,0.073,0.025
7,0.0000,0.01,0.10,0,0.00032,0.00000,0,0,0.0,0.0,...,0.255,0.001,0.000,0.002,0.00,0.00,0,0.000,0.000,0.000
8,0.0000,0.01,0.10,0,0.00004,0.00004,0,0,0.0,0.0,...,0.001,0.001,0.100,0.000,0.10,0.00,0,0.000,0.000,0.000
9,0.0000,0.01,0.10,0,0.00004,0.00000,0,0,0.0,0.0,...,0.002,0.002,0.100,0.000,0.05,0.00,0,0.000,0.000,0.000


Data_of_Attack_Back_Satan.csv:


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0.0000,0.02,0.06,0.0,0.00020,0.00000,0,0,0,0.0,...,0.143,0.001,0.001,0.004,0.001,0.0,0.001,0.0,0.0,0.0
1,0.0000,0.02,0.06,0.0,0.00020,0.00000,0,0,0,0.0,...,0.144,0.002,0.001,0.004,0.002,0.0,0.001,0.0,0.0,0.0
2,0.0000,0.02,0.06,0.0,0.00020,0.00000,0,0,0,0.0,...,0.145,0.003,0.002,0.004,0.003,0.0,0.001,0.0,0.0,0.0
3,0.0000,0.01,0.12,0.0,0.00001,0.00000,0,0,0,0.0,...,0.146,0.001,0.001,0.005,0.001,0.0,0.001,0.0,0.0,0.0
4,0.0000,0.01,0.12,0.0,0.00001,0.00000,0,0,0,0.0,...,0.147,0.002,0.001,0.005,0.001,0.0,0.001,0.0,0.0,0.0
5,0.0000,0.01,0.12,0.0,0.00001,0.00000,0,0,0,0.0,...,0.148,0.001,0.001,0.005,0.002,0.0,0.001,0.0,0.0,0.0
6,0.0000,0.01,0.12,0.0,0.00001,0.00000,0,0,0,0.0,...,0.149,0.002,0.001,0.005,0.003,0.0,0.001,0.0,0.0,0.0
7,0.0000,0.01,0.02,0.0,0.00001,0.00000,0,0,0,0.0,...,0.150,0.001,0.001,0.006,0.003,0.0,0.001,0.0,0.0,0.0
8,0.0000,0.00,0.04,0.0,0.00005,0.01886,0,0,0,0.0,...,0.151,0.003,0.002,0.006,0.001,0.0,0.001,0.0,0.0,0.0
9,0.0001,0.00,0.04,0.0,0.00006,0.01405,0,0,0,0.0,...,0.152,0.004,0.003,0.006,0.001,0.0,0.001,0.0,0.0,0.0


Data_of_Attack_Back_Smurf.csv:


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,0.02,0.09,0,0.01032,0,0,0.0,0,0,...,0.146,0.001,0.001,0.002,0.001,0,0.0,0,0.0,0
1,0,0.02,0.09,0,0.01032,0,0,0.0,0,0,...,0.147,0.002,0.001,0.002,0.001,0,0.0,0,0.0,0
2,0,0.02,0.09,0,0.01032,0,0,0.0,0,0,...,0.148,0.003,0.002,0.002,0.002,0,0.0,0,0.0,0
3,0,0.02,0.09,0,0.01032,0,0,0.0,0,0,...,0.149,0.004,0.003,0.002,0.003,0,0.0,0,0.0,0
4,0,0.02,0.09,0,0.01032,0,0,0.0,0,0,...,0.150,0.005,0.003,0.002,0.003,0,0.0,0,0.0,0
5,0,0.02,0.09,0,0.01032,0,0,0.0,0,0,...,0.151,0.006,0.004,0.002,0.004,0,0.0,0,0.0,0
6,0,0.02,0.09,0,0.01032,0,0,0.0,0,0,...,0.152,0.007,0.005,0.002,0.005,0,0.0,0,0.0,0
7,0,0.02,0.09,0,0.01032,0,0,0.0,0,0,...,0.153,0.008,0.005,0.002,0.005,0,0.0,0,0.0,0
8,0,0.02,0.09,0,0.01032,0,0,0.0,0,0,...,0.154,0.009,0.006,0.002,0.006,0,0.0,0,0.0,0
9,0,0.02,0.09,0,0.01032,0,0,0.0,0,0,...,0.155,0.010,0.006,0.002,0.006,0,0.0,0,0.0,0


In [40]:
# get some information
# print(dfs['Data_of_Attack_Back.csv'].columns)
# print(dfs['Data_of_Attack_Back.csv'].info())
# print(dfs['Data_of_Attack_Back.csv'].index)

In [47]:
display(dfs['Data_of_Attack_Back_FTPWrite.csv'])

,0.0026,0,0.07,0.3,0.00116,0.00451,0.4,0.5,0.6,0.2,...,0.001.2,0.001.3,0.1.5,0.21,0.1.6,0.22,0.23,0.24,0.25,0.26
0,0.0134,0,0.34,0,0.00100,0.39445,0,0,0.2,0.0,...,0.002,0.001,0.05,0.1,0.05,0.000,0,0,0,0
1,0.0000,0,0.14,0,0.00613,0.00000,0,0,0.0,0.0,...,0.001,0.084,0.10,0.0,0.10,0.002,0,0,0,0
2,0.0000,0,0.14,0,0.00000,0.00005,0,0,0.0,0.0,...,0.002,0.085,0.10,0.0,0.10,0.002,0,0,0,0
3,0.0032,0,0.07,0,0.00104,0.00449,0,0,0.0,0.2,...,0.001,0.001,0.10,0.0,0.10,0.000,0,0,0,0
4,0.0067,0,0.34,0,0.00157,0.02703,0,0,0.1,0.0,...,0.002,0.001,0.05,0.1,0.05,0.000,0,0,0,0
5,0.0000,0,0.14,0,0.00676,0.00000,0,0,0.0,0.0,...,0.001,0.004,0.10,0.0,0.10,0.050,0,0,0,0
6,0.0000,0,0.14,0,0.00000,0.00005,0,0,0.0,0.0,...,0.002,0.005,0.10,0.0,0.10,0.040,0,0,0,0


In [55]:
# correct column names are missing in the 'Data_of_Attack_Back_FTPWrite.csv' dataset
# set the new column names
dfs['Data_of_Attack_Back_FTPWrite.csv'] = dfs['Data_of_Attack_Back_FTPWrite.csv'].set_axis(
    dfs['Data_of_Attack_Back.csv'].columns, axis = 'columns',
    copy = False,
)
display(dfs['Data_of_Attack_Back_FTPWrite.csv'])

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0.0134,0,0.34,0,0.00100,0.39445,0,0,0.2,0.0,...,0.002,0.001,0.05,0.1,0.05,0.000,0,0,0,0
1,0.0000,0,0.14,0,0.00613,0.00000,0,0,0.0,0.0,...,0.001,0.084,0.10,0.0,0.10,0.002,0,0,0,0
2,0.0000,0,0.14,0,0.00000,0.00005,0,0,0.0,0.0,...,0.002,0.085,0.10,0.0,0.10,0.002,0,0,0,0
3,0.0032,0,0.07,0,0.00104,0.00449,0,0,0.0,0.2,...,0.001,0.001,0.10,0.0,0.10,0.000,0,0,0,0
4,0.0067,0,0.34,0,0.00157,0.02703,0,0,0.1,0.0,...,0.002,0.001,0.05,0.1,0.05,0.000,0,0,0,0
5,0.0000,0,0.14,0,0.00676,0.00000,0,0,0.0,0.0,...,0.001,0.004,0.10,0.0,0.10,0.050,0,0,0,0
6,0.0000,0,0.14,0,0.00000,0.00005,0,0,0.0,0.0,...,0.002,0.005,0.10,0.0,0.10,0.040,0,0,0,0


In [56]:
# combine all data into one data frame
combined_df = pd.DataFrame(columns = dfs['Data_of_Attack_Back.csv'].columns)

# loop
for key, value in dfs.items():
    # append the 'attack' column to each table
    value['attack'] = key.split('.')[0]

    # concat the value into 'combined_df'
    combined_df = pd.concat([combined_df, value])

In [57]:
# view the number of the data records
print(f'number of records = {combined_df.index.size} records')

number of records = 817550 records


In [60]:
# set new index starting from 0 to size of data
combined_df = combined_df.set_axis(
    [x for x in range(0, combined_df.index.size)], 
    axis = 'index',
    copy = False,
)

In [61]:
display(combined_df)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack
0,0.0,0,0,0.0,0.54540,0.08314,0,0,0,0.2,...,0.001,0.100,0,0.100,0,0.0,0.0,0.0,0.0,Data_of_Attack_Back
1,0.0,0,0,0.0,0.54540,0.08314,0,0,0,0.2,...,0.002,0.100,0,0.050,0,0.0,0.0,0.0,0.0,Data_of_Attack_Back
2,0.0,0,0,0.0,0.54540,0.08314,0,0,0,0.2,...,0.003,0.100,0,0.033,0,0.0,0.0,0.0,0.0,Data_of_Attack_Back
3,0.0,0,0,0.0,0.54540,0.08314,0,0,0,0.2,...,0.004,0.100,0,0.025,0,0.0,0.0,0.0,0.0,Data_of_Attack_Back
4,0.0,0,0,0.0,0.54540,0.08314,0,0,0,0.2,...,0.005,0.100,0,0.020,0,0.0,0.0,0.0,0.0,Data_of_Attack_Back
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817545,0.0,0.02,0.09,0.0,0.01032,0.00000,0,0.0,0,0.0,...,0.251,0.098,0.001,0.098,0,0.0,0.0,0.0,0.0,Data_of_Attack_Back_Smurf
817546,0.0,0.02,0.09,0.0,0.01032,0.00000,0,0.0,0,0.0,...,0.252,0.099,0.001,0.099,0,0.0,0.0,0.0,0.0,Data_of_Attack_Back_Smurf
817547,0.0,0.02,0.09,0.0,0.01032,0.00000,0,0.0,0,0.0,...,0.253,0.099,0.001,0.099,0,0.0,0.0,0.0,0.0,Data_of_Attack_Back_Smurf
817548,0.0,0.02,0.09,0.0,0.01032,0.00000,0,0.0,0,0.0,...,0.254,0.100,0.001,0.100,0,0.0,0.0,0.0,0.0,Data_of_Attack_Back_Smurf


In [32]:
# combined_df.columns